In [ ]:
import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import shutil
from glob import glob
import pandas_profiling as pp
import cv2
from google.cloud import storage
from kaggle_datasets import KaggleDatasets
from random import seed, randint, random, choice
from PIL import Image
import tensorflow_addons as tfa

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [ ]:
IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600)
im_size = IMSIZES[7]

load_dir = "/kaggle/input/plant-pathology-2021-fgvc8/"
df = pd.read_csv(load_dir + 'train.csv')

strategy = auto_select_accelerator()
BATCH_SIZE = 32

test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

n_labels = 5

In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = 32

test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

testgen = ImageDataGenerator(
            preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=(0.8, 1.2),
            rescale=1/255.0)

test_set = testgen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_dir,
                                    x_col="image",
                                    y_col=None,
                                    batch_size=BATCH_SIZE,
                                    seed=42,
                                    shuffle=False,
                                    class_mode=None,
                                    target_size=(im_size,im_size))

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, GlobalAveragePooling2D, ReLU, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D, GlobalMaxPooling2D
import tensorflow_addons

with strategy.scope():
    base = tf.keras.applications.EfficientNetB7(weights=None,
                                                include_top=False, 
                                                input_shape=(im_size, im_size, 3),
                                                drop_connect_rate=0.4)

    model_69 = Sequential()
    model_69.add(base)
    model_69.add(GlobalAveragePooling2D())
    model_69.add(Dense(n_labels, activation = 'sigmoid'))
model_69.load_weights('/kaggle/input/effnetb7-2/besteffb7_2.h5')

In [ ]:
with strategy.scope():
    base = tf.keras.applications.EfficientNetB7(weights=None,
                                                include_top=False, 
                                                input_shape=(im_size, im_size, 3),
                                                drop_connect_rate=0.4)
    model_71 = Sequential()
    model_71.add(base)
    model_71.add(GlobalAveragePooling2D())
    model_71.add(Dense(n_labels, activation = 'sigmoid'))
model_71.load_weights('/kaggle/input/effmodel1-2/besteffb7.h5')

In [ ]:
with strategy.scope():
    base = tf.keras.applications.EfficientNetB7(weights=None,
                                                include_top=False, 
                                                input_shape=(im_size, im_size, 3),
                                                drop_connect_rate=0.4)
    model_72 = Sequential()
    model_72.add(base)
    model_72.add(GlobalMaxPooling2D())
    model_72.add(Dense(n_labels, activation = 'sigmoid'))
model_72.load_weights('/kaggle/input/plant-1-model-2/bestplantmodel_1model.h5')

In [ ]:
TTA = 3 
preds = []

for i in range(TTA):
    test_set.reset()
    preds.append(model_69.predict(test_set))
    preds.append(model_71.predict(test_set))
    preds.append(model_72.predict(test_set))
    
pred = np.mean(np.array(preds), axis=0)

In [ ]:
name = {0: 'complex',
        1: 'scab',
        2: 'frog_eye_leaf_spot',
        3: 'rust',
        4: 'powdery_mildew',
        6: 'healthy'}

threshold = {0: 0.25,
             1: 0.6,
             2: 0.7,
             3: 0.5,
             4: 0.5}

def get_key(val):
    for key, value in name.items():
        if val == value:
            return key
 
    return "key doesn't exist"

pred_string = []
for line in pred:
    s = ''
    for i in range(n_labels):
        if line[i] > threshold[i]:
            s = s + name[i] + ' '
    
    if s == '': 
        s = name[6]
    pred_string.append(s)
    
test_df['labels'] = pred_string
test_df.to_csv('submission.csv', index=False)
test_df.head()

In [ ]:
# one_hot = {'healthy':            [0, 0, 0, 0, 0],
#            'complex':            [1, 0, 0, 0, 0],
#            'scab':               [0, 1, 0, 0, 0],
#            'frog_eye_leaf_spot': [0, 0, 1, 0, 0],
#            'rust':               [0, 0, 0, 1, 0],
#            'powdery_mildew':     [0, 0, 0, 0, 1],
#            'scab frog_eye_leaf_spot': [0, 1, 1, 0, 0],
#            'rust frog_eye_leaf_spot': [0, 0, 1, 1, 0],
#            'frog_eye_leaf_spot complex': [1, 0, 1, 0, 0],
#            'scab frog_eye_leaf_spot complex': [1, 1, 1, 0, 0],
#            'powdery_mildew complex': [1, 0, 0, 0, 1],
#            'rust complex': [1, 0, 0, 1, 0]}
pred